In [1]:
from transformers import ViTForImageClassification, AutoModelForImageClassification
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from torch.profiler import profile, record_function, ProfilerActivity
from tqdm.auto import tqdm
from torch.utils.data import Subset
import numpy as np

In [2]:
vit_model = ViTForImageClassification.from_pretrained(
    "Swapnil949/vit-finetuned-cifar100")
mamba_model = AutoModelForImageClassification.from_pretrained(
    "Swapnil949/mambavision-finetuned-cifar100",
    ignore_mismatched_sizes=True,
    trust_remote_code=True)


/root/miniconda3/envs/mamba/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/root/miniconda3/envs/mamba/lib/python3.10/site-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/root/miniconda3/envs/mamba/lib/python3.10/site-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/root/miniconda3/envs/mamba/lib/python3.10/site-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` i

In [3]:
trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
test_dataset = datasets.CIFAR100(root="./data", train=False, download=True, transform=trans)

# Choose a fraction (e.g., 10%) of the data
fraction = 0.001
num_samples = int(len(test_dataset) * fraction)
indices = np.random.choice(len(test_dataset), num_samples, replace=False)  # Randomly select indices

# Create a subset of the dataset
small_test_dataset = Subset(test_dataset, indices)

# Create a DataLoader for the smaller dataset
test_loader = DataLoader(small_test_dataset, batch_size=1, shuffle=False)                         


Files already downloaded and verified


In [8]:
from tqdm import tqdm

# Profile the inference for both models
for model_name, model in zip(["ViT", "MambaVision"], [vit_model, mamba_model]):
    print(f"Profiling {model_name} model...")
    model.eval()
    model.cuda()

    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        with record_function("model_inference"):
            for inputs, _ in test_loader:
                inputs = inputs.cuda()
                with torch.no_grad():
                    model(inputs)

    # Print profiling results
    print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))


Profiling ViT model...


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us     223.753ms        89.73%     223.753ms     223.753ms             1  
                                        model_inference        29.10%      68.800ms       100.00%     236.385ms     236.385ms       0.000us         0.00%      25.611ms      25.611ms             1  
         